In [1]:
import pymrio
exio3 = pymrio.parse_exiobase3(path="exiobase/IOT_2022_pxp.zip")
import country_converter as coco
cc = coco.CountryConverter()
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
exio3.calc_all()
exio3_ori = exio3.copy()
emptyY = exio3.Y.copy()
emptyY.iloc[:,:] = 0

x = exio3.x.copy()
Z = exio3.Z.copy()
A = exio3.A.copy()
L = exio3.L.copy()


In [3]:
# Setting up the variables
vehicles_list = ["ICEV","PHEV","BEV","FCEV"]
stressor = "Carbon dioxide (CO2) IPCC categories 1 to 4 and 6 to 7 (excl land use, land use change and forestry)"
scenario_list = ["Projection-APS"]
year_list = [2030]
demand_list = ['normal']
demand = demand_list[0]
column_list = ["scenario","year","vehicles","stage","region","sector","emission"]
stage = "manufacturing"
scenario = scenario_list[0]

In [4]:
def calc_emission(vehicles,outlook): 
    demandY = emptyY.copy()
    
    demandY["NL",vehicles] = outlook[vehicles] ## NL as a place holder for european demand, we donot concern on which country the EV will be consumed, only the aggregated EU demand
    demandY.fillna(0,inplace=True)
    
    exio3.Y = demandY
    
    # calculate with new demand
    # coeff : A, L, S, M should be the same with original
    exio3.x = None # reset the output
    exio3.Z = None  # expect new transaction to satisfy the new the output
    exio3.impacts.D_cba = None # reset foot print cba, others are not synch because they consider the emission from final demand
    exio3.impacts.F = None
    exio3.calc_all()
    
    diag_emission = None
    #become a matrix F of regional - sector X regional - sector 
    diag_emission = exio3.impacts.diag_stressor(stressor)
    diag_emission.F
    
    exio3.emission  = diag_emission
    exio3.calc_all()
    
    #print(exio3.meta)
    print("done calculation for "+vehicles)
    emission = exio3.emission.D_cba
    return emission

In [5]:
# read new outlook 
file_path = "data/outlook_"+stage+"_country_"+scenario+".csv"
if (stage=="manufacturing"):
   file_path = "data/outlook_"+stage+"_country_"+scenario+"_"+demand+".csv"

print(file_path)
#---------------------------------
outlook = pd.read_csv(file_path)
outlook.set_index(keys=["year","region","sector"],inplace=True)

data/outlook_manufacturing_country_Projection-APS_normal.csv


In [6]:
outlook.index.get_level_values("region").unique()

Index(['CN', 'CZ', 'DE', 'FR', 'IT', 'KR', 'RO', 'SE', 'US'], dtype='object', name='region')

In [7]:
#start collectimg emission data
emission_stage = pd.DataFrame(columns=column_list)

for year in year_list: 
    for vehicles in vehicles_list:
        emission = calc_emission(vehicles,outlook.loc[year])
        emission_regions_total = emission.sum(axis=1).to_frame().reset_index()
        emission_regions_total.rename(columns = {0:"emission"}, inplace=True)
        emission_regions_total["stage"] = stage
        emission_regions_total["scenario"] =  scenario
        emission_regions_total["year"] =  year
        emission_regions_total["vehicles"] =  vehicles
        
        ## add to overall record
        emission_stage = pd.concat([emission_stage,emission_regions_total],axis=0) 

emission_stage.reset_index(drop=True,inplace=True) 
emission_stage.to_csv("outputs/emission_by_stage_"+stage+"_"+demand+"_country.csv",index=False)

done calculation for ICEV
done calculation for PHEV
done calculation for BEV
done calculation for FCEV
